# Format de compression MPEG

## Etape 1 : coder une image en JPEG

On va effectuer une quantification des coeffs. On n'implémente pas la compression JPEG mais on peut simuler la compression JPEG. On se contente de calculer la place théorique que prend l'image -> décomposition en bloc 8x8 -> transformée en cosinus discrets

Librairies à utiliser : cv2, numpy et math

JPEG = Joint Photographic Experts Group

Le processus de compression JPEG comporte six étapes principales. On part de l'image brute :

1. Transformation de couleurs
2. Sous échantillonnage
3. Découpage en blocs de pixels
4. DCT
5. Quantification
6. Codage RLE et Huffman (on le fait pas nn?)

Et on obtient une image compressée selon JPEG.

Description des étapes :

On distingue les basses et hautes freq : les basses constituent les données majeures présentes dans une image. Les hautes freq caractérisent les zones à fort contraste (changement brusques de couleur). Les hautes freq sont moins visibles, c'est dessus que la compression s'effectuera.

- Quantification : là où se produite la majeure partie de la perte d'information et qui permet de gagner de la place. La quantification consiste à diviser cette mat de 8x8 (retournée par la DCT pour chaque bloc) par une matrice de quantification de 8x8 coeff choisis par le codeur. 

- Compression RLE et Huffman : le codage s'effectue en zigzag sur l'image. Ce résultat est compressé selon l'algorithme RLE (run length encoding, algo de compression sans pertes) basé sur la val 0 et un codage entropique de type Huffman (algo de compression sans perte basé sur un arbre)

### Réu du 4/11 :


simuler un codage jpeg : prgm python à partir d'une image png on reproduit l'image comme si elle était codée en jpeg. on la découpe en 8x8, on fait un cos discret, on quantifie les coeff avec la table de quantification de jpeg estimation du poids qu'elle prendrait si elle était codée en jpeg

calcul du poids : dépend de l'entropie des coefficients. l'entropie mesure la qtté d'info donné par les coeff
on prend une suite, un prg python qui calcule l'entropie de la suite 

un prg python qui à pt d'une image fait le codage jpeg, renvoie l'image et un nb (l'entropie)

pas de quantification qui soit pas trop petit
interpolation binaire cubique par ex pour les rotations (déjà des codes python qui existent)

## Codage compression JPEG :

On importe les librairies utiles

In [27]:
import numpy as np
import matplotlib.pyplot as plt

from math import cos, pi, sqrt
from copy import deepcopy
from typing import List
from PIL import Image

### Importation de l'image

Rappel : Une image est généralement codée sur trois canaux de couleurs : Rouge, Vert, Bleu (RGB).
Ainsi chaque pixel d'une image est défini par trois coefficients codés sur 8bits (le coefficient prend une valeur entre 0 et 255). Sans compression, une image de dimension 512x512 prend 512x512x8x3 = 6 291 456 bits = 786, 432 Ko.


In [26]:
#Load the image
img_RGB = Image.open('montagne.jpg')

#Get basic details about the image
print(img_RGB.format)
print(img_RGB.mode)
print(img_RGB.size)

#show the image
#plt.imshow(img_RGB)
#ou 
img_RGB.show() #, qui ouvre une fenêtre

JPEG
RGB
(4608, 3456)


La taille de l'image ici est, d'après le lecteur d'image, de 23.8 MB.

4608x 3456 x 8 x 3 = 381 791 232 bits = 47723,904 Ko



### 1.Transformation des couleurs

D'abort on va transformer les couleurs de l'image.
Les codages de couleur type luminance/chrominance donnent les meilleurs taux de compression car oeil humain assez sensible à la luminosité (luminance) mais peu à la teinte (chrominance) d'une image. On fera donc un sous échantillonnage de couleurs sur ces couleurs là plutôt que sur RGB.

![Perception](perception.png)

La figure montre que la sensibilité de l'oeil humain est bien différente pour les couleurs rouge, vert et bleue constitutives de nos images. Ainsi le vert est-il le mieux perçu, puis vient le rouge, et enfin le bleu de maniere minoritaire. On passera donc d'une image codée en RGB à une image codée en fonction de sa luminance (Y), et de sa chrominance (Cb, Cr) (format YUV)



Changer de couleurs RGB à YUV consiste à faire un changement de base orthogonale (Rappel : la base de RGB est orthogonale)

En principe, on a en quelque sorte :

  $$  Y ≃ R + G + B \\
    U ≃ B – Y \\
    V ≃ R – Y$$
    
La matrice de changement de base est plus particulièrement définie ainsi : (Wikipédia + autres sources)

![changement](changement_base.png)

Ici, on utilise simplement une fonction du module Image de la librairie PIL.

In [20]:
def RGB_to_YUV(img):
    img_YUV = img.convert('YCbCr')
    return img_YUV

In [22]:
img_YUV = RGB_to_YUV(img_RGB)
img_YUV.show()

### 2. Sous échantillonnage des couleurs

On doit donc en suite sous échantillonner les couleurs. On sépare les channels. On ne touche pas au channel Y mais on va rétrécir les image de U et V. Il y a plusieurs réglages possibles que l'on décrit avec la « notation J:a:b », définie ainsi, par bloc de 8x8 :
- J est le nombre de pixels de Y conservés pour 4 pixels affichés, sur chaque ligne ;
- a est le nombre de pixels de U conservés pour 4 pixels affichés, sur les lignes paires ;
- b est le nombre de pixels de V conservés pour 4 pixels affichés, sur les lignes impaires.
![subsampling](subsampling.png)

Ainsi le sous échantillonnage de couleur le plus utilisé est le 4:2:0 (c'est à dire qu'on découpe l'image en bloc de 8x8). (Mais ce n'est pas le plus important, on peut prendre du 4:4:4)


### 3. Découpage en blocs

En JPEG, on ne travaille pas sur une image entière : on travaille sur des blocs de 8x8 pixels (séparément en ce qui concerne l’intensité, le bleu et le rouge, donc).
Si la taille d’une image n’est pas exactement un multiple de 8 dans un axe donné, et que
la compression est forte, de légers défauts de compression pourraient apparaître. C’est un des soucis de JPEG.

Chaque bloc de 8x8 est en suite envoyé pour être transformé par DCT.



In [28]:
def chunkify(img, block_width=8, block_height=8):
    shape = img.shape
    x_len = shape[0]//block_width
    y_len = shape[1]//block_height
    #print(x_len, y_len)
    
    chunks = []
    x_indices = [i for i in range(0, shape[0]+1, block_width)]
    y_indices = [i for i in range(0, shape[1]+1, block_height)]

    shapes = list(zip(x_indices, y_indices))
    
    for i in range(len(shapes)):
        try:
            start_x = shapes[i][0]
            start_y = shapes[i][1]
            end_x = shapes[i+1][0]
            end_y = shapes[i+1][1]
            chunks.append( shapes[start_x:end_x][start_y:end_y] )
        except IndexError:
            print('End of Array')

    return chunks

### 4. Transformée en cosinus discret 

On fait une transformée DCT soit Discrete Cosine Transform. On applique cette transfo numérique à chaque bloc (variante de la transfo de fourier). Cette transfo décompose un bloc (considéré comme une fc num à deux variables) en une somme de fc cosinus oscillant à des freq différentes. Chaque bloc est ainsi décrit en une carte de freq et en amplitude plutôt qu'en puexels et coeff de couleur. (formule de la DCT dispo sur wiki) Le calcul d'une DCT est l'étape qui coûte le plus de temps et de ressources dans la compression JPE. Mais elle peremt de séparer les basses et hautes freq de l'image. 

In [ ]:
"""
    On part d'un tableau de valeurs y, x (ligne, colonne) et on en
    renvoie un nouveau.
"""

def encoder_dct(ancien_tableau : List[List[int]]) -> List[List[int]]:
    
    # Le nouveau tableau aura le même nombre d'entrées (et de sous-
    # entrées) que le tableau d'origine, mais pas les mêmes valeurs,
    # c'est pourquoi on commence à copier le tableau d'origine (et
    # ses tableaux imbriqués) pour en créer un distinct.
    
    nouveau_tableau : List[List[int]] = deepcopy(ancien_tableau)
    
    for nouveau_y in range(8):
        for nouveau_x in range(8):
            
            nouvelle_valeur = 0
            
            for ancien_y in range(8):
                for ancien_x in range(8):
                    
                    # Le cosinus retourne un facteur qui pondère
                    # la valeur selon si on est dans la strie ou non.

                    nouvelle_valeur += (
                        ancien_tableau[ancien_y][ancien_x] *
                        cos(((2 * ancien_y + 1) * nouveau_y * pi) / 16) *
                        cos(((2 * ancien_x + 1) * nouveau_x * pi) / 16)
                    )
            
            # Si on est au bord du tableau (aplat complet),
            # cosinus retournera toujours 1 et le nombre
            # pourrait être très gros : on va donc le
            # réduire un peu
            
            if nouveau_y == 0:
                nouvelle_valeur /= sqrt(2)
            if nouveau_x == 0:
                nouvelle_valeur /= sqrt(2)
            nouvelle_valeur /= 4
            
            nouveau_tableau[nouveau_y][nouveau_x] = nouvelle_valeur
    
    return nouveau_tableau

Sources :

http://www-ljk.imag.fr/membres/Valerie.Perrier/SiteWeb/node8.html

https://fr.wikipedia.org/wiki/YUV

https://compression.fiches-horaires.net/la-compression-avec-perte-1/le-compression-jpeg/

https://github.com/QuantumNovice/ImageProcessing/blob/master/image_chunkify.py

DCT :

http://www-ljk.imag.fr/membres/Valerie.Perrier/SiteWeb/node9.html
